# High Level Overview

Examine variable importance of drivers to loyalty for segments/cohorts (instead of ALL DATA, as was used in the UX Index calculation). Ultimately, this taps into segment/cohort driven differences in UXIndex and UX_Loyalty. UX_Loyalty is an individual metric, calculated directly from individual answeres to questions. UXIndex was calculated based on weighting of drivers with the weights determined based on variable importance to UX_Loyalty at an aggregate level ( for ALL DATA) (refered to as **universal weights**). As such, we imagine tht for some cohorts/segements where the aggregate mapping does not match, UX_index would be a worse approximationthan for cohorts segemnts where the aggregate mapping does match.

In [1]:
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline 

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans


from sklearn.linear_model import LogisticRegression

import scipy

In [3]:
pd.set_option('display.max_columns', 1000)
pd.options.display.max_rows=1000

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

plt.rcParams.update({'figure.max_open_warning': 0})

In [2]:
root = r"../data/input/07 Samsung UX Index - Web App Implementation/"

fname_data = root + r"Samsung UX Index Survey_Data.csv"
df_data = pd.read_csv(fname_data)

fname_vaxmap = root + r"Samsung UX Index Survey_Datamap.xlsx"
df_varmap = pd.read_excel(fname_vaxmap, header=1, sheet_name=0)
df_valmap = pd.read_excel(fname_vaxmap, header=1, sheet_name=1)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1218) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 0A. Confirm Loyalty Calc

There are two advocacy metrics in the survey. We calculated the mean value of the two metrics for all survey respondents.
The calculated advocacy value was then averaged with the other two loyalty metrics: retention and enrichment.
These values were then used to create a Loyalty Index value for all individual survey respondents.
Each respondent in the dataset was assigned a Loyalty Index score (this is stored in the “UX_Loyalty” variable).

In [6]:
cols_loyalty = ['qxadvocacy01_1', 'qxadvocacy02_1', 'qxretention_1', 'qxenrichment_1','UX_Loyalty']
df_loyalty =  df_data[cols_loyalty]

In [7]:
df_loyalty.head()

,qxadvocacy01_1,qxadvocacy02_1,qxretention_1,qxenrichment_1,UX_Loyalty
0,4,4,4,4,4.000000
1,6,5,4,5,4.833333
2,6,6,4,6,5.333333
3,6,4,6,5,5.333333
4,7,1,5,7,5.333333


In [16]:
driver_calc = [((df_loyalty.qxadvocacy01_1+df_loyalty.qxadvocacy02_1)/2 +
           df_loyalty.qxretention_1+df_loyalty.qxenrichment_1)/3] 

driver_calc

[0       4.000000
 1       4.833333
 2       5.333333
 3       5.333333
 4       5.333333
           ...   
 3249    5.500000
 3250    5.500000
 3251    7.000000
 3252    5.333333
 3253    7.000000
 Length: 3254, dtype: float64]

In [20]:
df_loyalty.UX_Loyalty.to_list()[-10:]

[3.333333333, 7.0, 7.0, 7.0, 7.0, 5.5, 5.5, 7.0, 5.333333333, 7.0]

# 0B. Confirm UXIndex Calc

Shapley Value Regression derives the relationship between the quality indicators (feeling, trust, simplicity and function) and the Loyalty Index. This relationship is represented by the relative importance scores that sum up to 100%.  These scores are used as universal weights for the UX Index calculation throughout the survey (including the UX Index scores of different activities). 

Using the weighting from previous steps, individual survey responses to the four quality indicator questions were transformed into a single UX Index score for each survey respondent. (Note: Relative Importance coefficients are the same for each respondents, while Quality Indicator scores are different for each respondent). Weighting was used in order to assign more weight to more important quality indicators. The following algorithm was used:
Transform the quality indicator question responses from 1-7-point scale to 0-100 scale using the following formula: (x-1)x(100/6) 

The 0-100 scores for each quality indicator were multiplied by the quality indicator weight from the first step.

The squared score from the previous step, divided by 100, was used as a final UX Index score. This adjustment was made in order to provide a better differentiation across the UXI scores for different respondent groups, since a large proportion of responses in the quality indicator questions were focused in the “Top 3” bucket (i.e., a 5, 6 or 7 on the seven-point scale).
